# 9장 향후 판매량 예측 경진대회 환경 세팅된 노트북 양식

In [ ]:
import pandas as pd
sales_train = pd.read_csv('')

In [ ]:
#데이터 합치기 381
train = sales_train.merge(shops, on='shop_id', how='left')
train = train.merge(items, on='item_id', how='left')
train = train.merge(item_categories, on='item_category_id', how='left')
#train.head()

In [ ]:
#요약표 383

# 데이터 시각화 384
# 일 판매량
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
#sns.boxplot(data=train, y='item_cnt_day')

# 판매가격
#sns.boxplot(data=train, y='item_price')

# 그룹화
group = train.groupby('date_block_num').agg({'item_cnt_day':'sum'})
group.reset_index()


mpl.rc('font', size=13)
figure, ax = plt.subplots()
figure.set_size_inches(11, 5)
# 월별판매량
group_month_sum = train.groupby('date_block_num').agg({'item_cnt_day':'sum'})
group_month_sum = group_month_sum.reset_index()

sns.barplot(data=group_month_sum, x='date_block_num', y='item_cnt_day')
ax.set(title='Distribution of monthly item counts by data block number', xlabel='Date block number', ylabel='Monthly item counts')

# 상품별 판매량
figure, ax = plt.subplots()
figure.set_size_inches(11,5)
group_cat_sum = train.groupby('item_category_id').agg({'item_cnt_day':'sum'})
group_cat_sum = group_cat_sum.reset_index()
group_cat_sum = group_cat_sum[group_cat_sum['item_cnt_day'] > 10000]

sns.barplot(data=group_cat_sum, x='item_category_id', y='item_cnt_day')
ax.set(title='Distribution of monthly item counts by item category id',
       xlabel='Date block number', ylabel='total item counts')

# 상점별 판매량
figure, ax = plt.subplots()
figure.set_size_inches(11,5)
group_shop_sum = train.groupby('shop_id').agg({'item_cnt_day':'sum'})
group_shop_sum = group_shop_sum.reset_index()
group_shop_sum = group_shop_sum[group_shop_sum['item_cnt_day'] > 10000]

sns.barplot(data=group_shop_sum, x='shop_id', y='item_cnt_day')
ax.set(title='Distribution of total item counts by shop id',
       xlabel='shop ID', ylabel='total item counts')
ax.tick_params(axis='x', labelrotation=90)


In [ ]:
shops = shops.rename(columns={'shop_name':'상점명', 'shop_id':'상점ID'})
#shops.head()
items = items.rename(columns={'item_name':'상품명', 'item_id':'상품ID', 'item_category_id':'상품분류ID'})
#items.head()
item_categories = item_categories.rename(columns={'item_category_name':'상품분류명', 'item_category_id':'상품분류ID'})
#item_categories.head()
test = test.rename(columns={'shop_id':'상점ID', 'item_id':'상품ID'})
#test.head()

In [ ]:
# 작은 데이터 타입으로 변경하기 396
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('{:.1f}% 압축 됨'.format(100 * (start_mem - end_mem) / start_mem))
        
    return df


all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

In [ ]:
# 데이터 조합 398 꼭 앞 글을 읽어주세요.
from itertools import product

train=[]
for i in sales_train['월ID'].unique(): # 하나씩 추출
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID','상점ID','상품ID']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train

In [ ]:
# 항목 월간 판매량 추가하기 399 
group = sales_train.groupby(idx_features).agg({'판매량': 'sum'})
group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량'})
group

In [ ]:
# 2025년 10월 데이터 만들기
test['월ID'] = 34

#데이터 붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features
                    )

#Nan -> 0
all_data = all_data.fillna(0)
#all_data.head()

In [ ]:
#최종 403

all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')
all_data = downcast(all_data)
del shops, items, item_categories
gc.collect()

In [ ]:
all_data = all_data.drop(['상점명','상품명','상품분류명'], axis=1)

In [ ]:
# 훈련하기 404
x_train = all_data[all_data['월ID'] < 33]
x_train = x_train.drop(['월간 판매량'], axis=1)

x_valid = all_data[all_data['월ID'] == 33]
x_valid = x_valid.drop(['월간 판매량'], axis=1)

x_test = all_data[all_data['월ID'] == 34]
x_test = x_test.drop(['월간 판매량'], axis=1)

y_train = all_data[all_data['월ID'] < 33 ]['월간 판매량']
y_train = y_train.clip(0, 20)

y_valid = all_data[all_data['월ID'] == 33 ]['월간 판매량']
y_valid = y_valid.clip(0, 20)

In [ ]:
# 훈련시키기
import lightgbm as lgb
params = {'metric':'rmse', 
          'num_leaves':255, 
          'learning_rate':0.01, 
          'force_col_wise':True, 
          'random_state':10
         }

cat_features = ['상점ID', '상품분류ID']

dtrain = lgb.Dataset(x_train, y_train)
dvalid = lgb.Dataset(x_valid, y_valid)

#lightgbm
lgb_model = lgb.train(params = params,
                      train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      verbose_eval=50
    
)

In [ ]:
# 409
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories= pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
test= pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
sales_train = sales_train.rename(columns={'date':'날짜',
                                          'date_block_num':'월ID', 
                                          'shop_id':'상점ID',
                                          'item_id':'상품ID', 
                                          'item_price':'판매가',
                                          'item_cnt_day':'판매량'
                                         })
shops = shops.rename(columns={'shop_name':'상점명', 'shop_id':'상점ID'})
items = items.rename(columns={'item_name':'상품명', 'item_id':'상품ID', 'item_category_id':'상품분류ID'})
item_categories = item_categories.rename(columns={'item_category_name':'상품분류명', 'item_category_id':'상품분류ID'})
test = test.rename(columns={'shop_id':'상점ID', 'item_id':'상품ID'})

In [ ]:
# 작은 데이터 타입으로 변경하기 410
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('{:.1f}% 압축 됨'.format(100 * (start_mem - end_mem) / start_mem))
        
    return df


all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

In [ ]:
# 전처리 작업하기 411
sales_train = sales_train[sales_train['판매가'] > 0]
sales_train = sales_train[sales_train['판매가'] < 50000]

sales_train = sales_train[sales_train['판매량'] > 0]
sales_train = sales_train[sales_train['판매량'] < 1000]

In [ ]:
# 상점명 수정하기 412
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

#도시명으로 변경하기 413
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])
# shops['도시'].unique()
shops.loc[shops['도시'] == '!Якутск', '도시'] = 'Якутск'
# shops['도시'].unique()

In [ ]:
# 414
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
shops['도시'] = label_encoder.fit_transform(shops['도시'])

# 상점명 피처 제거하기 = 한 번만
shops = shops.drop('상점명', axis=1)
# shops.head()

# 상품명 제거 = 한 번만
items = items.drop('상품명', axis=1)
# items.head()

In [ ]:
# 415
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID':'min'})['월ID']
# 결측치 제거하기 
items['첫 판매월'] = items['첫 판매월'].fillna(34)
# 결측치 확인하기
# items[items['첫 판매월'].isna()]

# items.head()

In [ ]:
# 상품 분류명의 첫 단어를 대분류로 추출하기 417
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])
item_categories['대분류'].value_counts()

In [ ]:
# 인코딩 418
def make_etc(x):
    if len(item_categories[item_categories['대분류'] == x]) >= 5:
        return x
    else :
        return 'etc'

item_categories['대분류'] = item_categories['대분류'].apply(make_etc)
#item_categories.head()

In [ ]:
# 상품분류명 제거하기 419
item_categories = item_categories.drop('상품분류명', axis=1)

In [ ]:
# 데이터 합치기 419
from itertools import product

train = []
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID','상품ID']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [ ]:
# 420
group = sales_train.groupby(idx_features).agg({'판매량':'sum','판매가':'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량','판매가':'평균 판매가'})
train = train.merge(group, on=idx_features, how='left')
# train.head()

import gc
del group
gc.collect()

In [ ]:
# 피처 만들기 421
group = sales_train.groupby(idx_features).agg({'판매량':'count'})
group = group.reset_index()
group = group.rename(columns={'판매량':'판매건수'})
train = train.merge(group, on=idx_features, how='left')

del group, sales_train
gc.collect()
train.head()

In [ ]:
# 이어붙이기 422
test['월ID'] = 34

all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
all_data = all_data.fillna(0)
all_data.head()

In [ ]:
# 나머지 데이터 병합하기 422~423
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

all_data = downcast(all_data)

In [ ]:
# 기준 피처별 월간 평균 판매량 파생 피처 생성하기 424
def add_mean_features(df, mean_features, idx_features):
    assert(idx_features[0] == '월ID') and len(idx_features) in [2,3]

    if len(idx_features) == 2:
        features_name = idx_features[1] + '별 평균 판매량'
    else:
        features_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'

    group = df.groupby(idx_features).agg({'월간 판매량':'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량':features_name})

    df = df.merge(group, on=idx_features, how='left')
    df = downcast(df, verbose=False)
    mean_features.append(features_name)
    del group
    gc.collect()
    return df, mean_features

In [ ]:
# 426
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
item_mean_features = []

# ['월ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성하기
all_data, item_mean_features = add_mean_features(df=all_data,
                                                mean_features=item_mean_features,
                                                idx_features=['월ID', '상품ID'])

# ['월ID', '상품ID', '도시']로 그룹화 한 월간 평균 판매량 파생 피처 생성하기
all_data, item_mean_features = add_mean_features(df=all_data,
                                                mean_features=item_mean_features,
                                                idx_features=['월ID', '상품ID', '도시'])


item_mean_features

In [ ]:
# 427
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 미처명을 담을 리스트
shop_mean_features = []

# ['월ID', '상점ID', '상품분류ID'] 로 그룹 화 한 월간 평균 판매량 파생 피처 생성하기
all_data, shop_mean_features = add_mean_features(df=all_data,
                                                mean_features=shop_mean_features,
                                                idx_features=['월ID', '상점ID', '상품분류ID']
                                                )
shop_mean_features

In [ ]:
# 시차 피처 생성하기 429
def add_lag_features(df, lag_features_to_clip, idx_features,
                        lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 데이터 프레임 부분만 복사해오기
    df_temp = df[idx_features + [lag_feature]].copy()
    #시차 피처 생성
    for i in range(1, nlags+1):
        #시차 피처이름
        lag_feature_name = lag_feature + '_시차' + str(i)
        #df_temp 열 이름 생성하기
        df_temp.columns = idx_features + [lag_feature_name]
        #df_temp의 date_block_num 피처에 +1
        df_temp['월ID'] += 1
        #idx_features를 기준으로 df와 df_temp 합치기
        df = df.merge(df_temp.drop_duplicates(),
                     on=idx_features,
                      how='left')
        # 결측값에 대해서 0으로 변경
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        if clip:
            lag_features_to_clip.append(lag_feature_name)
    df = downcast(df, False)    
    del df_temp
    gc.collect()

    return df, lag_features_to_clip

In [ ]:
# 위 함수 사용해보기
lag_features_to_clip = []
idx_features=['월ID','상점ID','상품ID']
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features= idx_features,
                                                 lag_feature='월간 판매량',
                                                 nlags=3,
                                                 clip=True)

In [ ]:
# 시차 피처 만들기
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features= idx_features,
                                                 lag_feature='판매건수',
                                                 nlags=3)

all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features= idx_features,
                                                 lag_feature='평균 판매가',
                                                 nlags=3)

In [ ]:
# 433
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성하기

for item_mean_feature in item_mean_features:
        all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features= idx_features,
                                                 lag_feature=item_mean_feature,
                                                 nlags=3,
                                                 clip=True)

all_data = all_data.drop(item_mean_features, axis=1)

In [ ]:
# 434
for shop_mean_feature in shop_mean_features:
        all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features= ['월ID','상점ID','상품분류ID'],
                                                 lag_feature=shop_mean_feature,
                                                 nlags=3,
                                                 clip=True)

all_data = all_data.drop(shop_mean_features, axis=1)

In [ ]:
# 결측치 처리 345
all_data = all_data.drop(all_data[all_data['월ID'] < 3 ].index)

In [ ]:
# 0~20 사이로 조정하기
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] =\
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0,20)

In [ ]:
# 정리하기 436~437
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

all_data['월'] = all_data['월ID'] % 12

all_data = all_data.drop(['첫 판매월','평균 판매가','판매건수'], axis=1)

all_data = downcast(all_data, False)

In [ ]:
x_train = all_data[all_data['월ID'] < 33]
x_train = x_train.drop(['월간 판매량'], axis=1)

x_valid = all_data[all_data['월ID'] == 33]
x_valid = x_valid.drop(['월간 판매량'], axis=1)

x_test = all_data[all_data['월ID'] == 34]
x_test = x_test.drop(['월간 판매량'], axis=1)

y_train = all_data[all_data['월ID']< 33]['월간 판매량']
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

del all_data
gc.collect()